# Handwriting recognition (HWR)

### Author: Mohammed Shehab

This script provides a comprehensive guide to:
1. Loading and preprocessing the dataset
2. Constructing a Convolutional Neural Network (CNN) model architecture
3. Performing hyperparameter tuning to optimize the CNN model
4. Exporting the best-performing model for deployment

**Deployment will be facilitated using Docker containerization to ensure a scalable and reproducible environment.**


In [1]:
!pip install mlflow

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
   ---------------------------------------- 0.0/26.7 MB ? eta -:--:--
    --------------------------------------- 0.5/26.7 MB 14.9 MB/s eta 0:00:02
   -- ------------------------------------- 1.4/26.7 MB 18.1 MB/s eta 0:00:02
   ---- ----------------------------------- 3.0/26.7 MB 23.7 MB/s eta 0:00:02
   ----- ---------------------------------- 3.6/26.7 MB 20.9 MB/s eta 0:00:02
   ------ --------------------------------- 4.6/26.7 MB 21.0 MB/s eta 0:00:02
   --------- ------------------------------ 6.1/26.7 MB 22.9 MB/s eta 0:00:01
   ----------- ---------------------------- 7.6/26.7 MB 24.2 MB/s eta 0:00:01
   -------------- ------------------------- 9.5/26.7 MB 26.5 MB/s eta 0:00:01
   --------------- ------------------------ 10.6/26.7

In [2]:
import mlflow
import mlflow.tensorflow


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from keras_tuner import HyperModel, RandomSearch
import numpy as np

Check GPU availablilty 

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


### Load dataset

In [5]:
# Load and preprocess the MNIST dataset
def load_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize
    x_train = np.expand_dims(x_train, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)
    return (x_train, y_train), (x_test, y_test)

### Define a HyperModel class for Keras Tuner with adjustable hyperparameters

In [6]:
class CNNHyperModel(HyperModel):
    def build(self, hp):
        model = models.Sequential()
        
        # Conv1
        model.add(layers.Conv2D(
            filters=hp.Int('conv1_filters', min_value=16, max_value=64, step=16),
            kernel_size=(3, 3),
            activation='relu',
            input_shape=(28, 28, 1)
        ))
        model.add(layers.MaxPooling2D((2, 2)))
        
        # Conv2_1 and Conv2_2
        for i in range(2):
            model.add(layers.Conv2D(
                filters=hp.Int(f'conv2_filters_{i+1}', min_value=32, max_value=128, step=32),
                kernel_size=(3, 3),
                activation='relu'
            ))
        
        # MaxPooling
        model.add(layers.MaxPooling2D((2, 2)))
        
        # Conv3_1 and Conv3_2
        model.add(layers.Conv2D(
            filters=hp.Int('conv3_filters', min_value=128, max_value=512, step=128),
            kernel_size=(3, 3),
            activation='relu'
        ))
        
        model.add(layers.Flatten())
        
        # Fully Connected Layer 1
        model.add(layers.Dense(
            units=hp.Int('fc1_units', min_value=500, max_value=1000, step=250),
            activation='relu'
        ))
        
        # Fully Connected Layer 2
        model.add(layers.Dense(
            units=hp.Int('fc2_units', min_value=250, max_value=500, step=250),
            activation='relu'
        ))
        
        # Output Layer
        model.add(layers.Dense(10, activation='softmax'))
        
        # Compile model
        model.compile(
            optimizer=tf.keras.optimizers.Adam(
                learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
            ),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        return model

In [8]:
(x_train, y_train), (x_test, y_test) = load_data()

In [9]:
x_train.shape

(60000, 28, 28, 1)

In [10]:
x_test.shape

(10000, 28, 28, 1)

### Initialize the Keras Tuner

In [11]:
tuner = RandomSearch(
    CNNHyperModel(),
    objective='val_accuracy',
    max_trials=10,  # Number of different hyperparameter combinations to try
    executions_per_trial=2,  # Average results over multiple runs
    directory='tuning_results',
    project_name='mnist_cnn_tuning'
)

Reloading Tuner from tuning_results\mnist_cnn_tuning\tuner0.json


### Perform hyperparameter search

In [15]:
# tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))
with mlflow.start_run():
    # Perform hyperparameter search
    tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))
    
    # Retrieve the best model and hyperparameters
    best_model = tuner.get_best_models(num_models=1)[0]
    best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
    
    # Log the best hyperparameters
    for param, value in best_hyperparameters.values.items():
        mlflow.log_param(param, value)
    
    # Retrieve the best trial's validation accuracy
    best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]
    best_val_accuracy = best_trial.score
    mlflow.log_metric("best_val_accuracy", best_val_accuracy)
    
    print("Best hyperparameters found:")
    print(best_hyperparameters.values)



Best hyperparameters found:
{'conv1_filters': 48, 'conv2_filters_1': 96, 'conv2_filters_2': 96, 'conv3_filters': 256, 'fc1_units': 1000, 'fc2_units': 250, 'learning_rate': 0.0003031375640856382}


### Export model for deployment phase

In [16]:
import os
from tensorflow.keras.models import save_model

# Save the best model for deployment
model_version = "v1_0_1"  # Update this version number as needed
model_directory = './exported_models/'
model_file_path = os.path.join(model_directory, f'best_model_{model_version}.h5')

os.makedirs(model_directory, exist_ok=True)
best_model.save(model_file_path)

# Log the model with MLflow
mlflow.tensorflow.log_model(best_model, artifact_path="model")
mlflow.log_param("model_version", model_version)
mlflow.end_run()

print(f"Model saved to {model_file_path}")



2024/10/11 13:14:31 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\Shehab\AppData\Local\Temp\tmpa7sga67a\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\Shehab\AppData\Local\Temp\tmpa7sga67a\model\data\model\assets
2024/10/11 13:14:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model saved to ./exported_models/best_model_v1_0_1.h5


### To access the MLflow UI, run !mlflow ui in a terminal, and open http://localhost:5000 in a browser.

In [ ]:
# !mlflow ui